In [1]:
import os
import numpy as np
import pandas as pd

import plotly.graph_objects as go

In [2]:
from plotly.offline import init_notebook_mode, iplot
init_notebook_mode(connected=True)

In [3]:
eval_scores = np.load('./data/eval_scores.npy', allow_pickle=True).item()
thres_scores = eval_scores['thres_scores']

In [4]:
auc = eval_scores['auc_score']
# fpr == 1-tnr(specificity)
fpr = [(1 - v['specificity']) for v in thres_scores.values()]
# tpr == recall
recall = [v['recall'] for v in thres_scores.values()]
thres = list(thres_scores.keys())

### ROC curve

### This plot shows
- ROC curve with filled area.
- Add a 45-degree line as a reference.

#### Main trace
```python
trace = go.Scatter(
    x=fpr,
    y=tpr,
    mode='lines',
    fill='tozeroy',
    line={'shape':'hv'},
    name='roc curve',
)
```
- fill: set the area to fill with a solid color.
- line: define line propertirs. `shape: 'hv'` means step-wise line shape.

#### Layout: shape
Reference line.
```python
shape = dict(
    type='line', 
    line=dict(dash='dash'),
    x0=0,
    x1=1,
    y0=0, 
    y1=1
)
```

In [5]:
def plot_roc_curve(fpr, tpr, auc=None, width=882, height=725):
    hovertemplate = (
        'FPR: %{x:,.4f} <br>' + 
        'TPR: %{y:,.4f}' + 
        '<extra></extra>'
    )
    
    if auc is not None:
        name = f'auc = {auc:.3f}'
    else:
        name = 'roc curve'
    
    fig = go.Figure()
    fig.add_trace(
        go.Scatter(x=fpr,
                   y=tpr,
                   mode='lines',
                   fill='tozeroy',
                   line={'shape':'hv'},
                   name=name,
                  )
    )
    
    
    fig.update_xaxes(title={'text':'False Positive Rate', 'standoff': 50, 'font':{'size': 14}})
    fig.update_yaxes(title={'text':'True Positive Rate', 'standoff':50, 'font':{'size': 14}}, ticksuffix=' ')
    fig.add_shape(
        type='line', line=dict(dash='dash'),
        x0=0, x1=1, y0=0, y1=1
    )
    fig.update_traces(hovertemplate=hovertemplate)
    fig.update_layout(
        title={
            'text': "ROC curve",    
            'y':0.99,
            'x':0,
            'xanchor': 'left',
            'yanchor': 'top',
            'font': {'size': 22},
        },
        showlegend=True,
        width=882,
        height=725,
        margin=dict(t=100, b=0, l=0, r=0),
        hoverlabel=dict(
            bgcolor="white",
            font_size=13,
            font_family="Rockwell",
        ),
        paper_bgcolor='rgba(0,0,0,0)',
        plot_bgcolor='rgba(238,238,238,1)',
        modebar=dict(
            bgcolor='rgba(0,0,0,0)', activecolor='rgba(68,68,68,0.7)', color='rgba(68,68,68,0.3)',
            remove=['zoom', 'lasso', 'select'],
        ),
    )

    return fig

In [6]:
fig = plot_roc_curve(fpr, recall, auc=auc)
fig.show(config={'displaylogo':False})
# fig.write_html('./automl_plot/roc_curve.html', config={'displaylogo':False}, include_plotlyjs='cdn', full_html=False)